In [1]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#  install.packages("BiocManager")
#BiocManager::install("iterativeBMA")
#install.packages("PDtoolkit")
# install.packages("remotes")
# remotes::install_github("ayhandis/creditR")

##########
# There doesn't appear to be an R implementation of the Spiegelhalter test.
# The SAS implementation seems to be very limited in its scope: it doesn't let you
# specify the hypothesized success probabilities as far as I can tell.

In [2]:
library(PDtoolkit, quietly=T)

options(warn=-1)

Warning message:
"replacing previous import 'lifecycle::last_warnings' by 'rlang::last_warnings' when loading 'tibble'"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'Hmisc'


The following objects are masked from 'package:dplyr':

    src, summarize


The following objects are masked from 'package:base':

    format.pval, units



Attaching package: 'PDtoolkit'


The following object is masked from 'package:stats':

    power




In [3]:
# Import data
test_data_main <- read.csv("pd_test_data_main.csv")
test_data_period_2 <- read.csv("pd_test_data_period_2.csv")

In [4]:
# Aggregate the results
bin_data <- test_data_main %>%
            group_by(ratings) %>%
            summarise_at(vars(predicted_pd, default_flag), funs(mean(.), sum(.), n()))

bin_data

ratings,predicted_pd_mean,default_flag_mean,predicted_pd_sum,default_flag_sum,predicted_pd_n,default_flag_n
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
A,0.10342906,0.13546798,41.9922,55,406,406
B,0.09952396,0.08541667,47.7715,41,480,480
C,0.09571932,0.08771930,10.9120,10,114,114


### 1. Binomial / Jeffrey's / Hoshmer-Lemeshow / z-score

In [5]:


# Test with PDtoolkit
pp.testing(rating.label = bin_data$ratings,
           pdc = bin_data$predicted_pd_mean,
           no = bin_data$predicted_pd_n,
           nb = bin_data$default_flag_sum, 
           alpha = 0.05)

rating,no,nb,odr,pdc,alpha,binomial,binomial.res,jeffreys,jeffreys.res,zscore,zscore.res,hosmer.lemeshow,hosmer.lemeshow.res
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
A,406,55,0.13546798,0.10342906,0.05,0.02389227,H1: ODR > PDC,0.01995857,H1: ODR > PDC,0.01700479,H1: ODR > PDC,0.13025,H0: ODR <= PDC
B,480,41,0.08541667,0.09952396,0.05,0.86744061,H0: ODR <= PDC,0.84955196,H0: ODR <= PDC,0.84906675,H0: ODR <= PDC,0.13025,H0: ODR <= PDC
C,114,10,0.08771930,0.09571932,0.05,0.66055279,H0: ODR <= PDC,0.59864873,H0: ODR <= PDC,0.61421823,H0: ODR <= PDC,0.13025,H0: ODR <= PDC


### 2. Brier Score

In [6]:
library(iterativeBMA, quietly = T)

brier.score(bin_data$predicted_pd_mean, bin_data$default_flag_mean)


Attaching package: 'robustbase'


The following object is masked from 'package:survival':

    heart


Scalable Robust Estimators with High Breakdown Point (version 1.7-1)



Attaching package: 'BiocGenerics'


The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")

[1] 0.001289508

### 3. Herfindahl

In [7]:
library(creditR, quietly = T)

h <- Herfindahl.Hirschman.Index(bin_data, "default_flag_n")
h


Attaching package: 'MLmetrics'


The following object is masked from 'package:base':

    Recall



Attaching package: 'creditR'


The following object is masked from 'package:PDtoolkit':

    scaled.score




# A tibble: 3 x 10
  ratings predicted_pd_mean default_flag_mean predicted_pd_sum default_flag_sum
  <chr>               <dbl>             <dbl>            <dbl>            <int>
1 A                  0.103             0.135              42.0               55
2 B                  0.0995            0.0854             47.8               41
3 C                  0.0957            0.0877             10.9               10
# ... with 5 more variables: predicted_pd_n <int>, default_flag_n <int>,
#   SumTotal <int>, concentration <df[,1]>, HHI <df[,1]>


[1] 0.408232

### 4. Spiegelhalter

In [8]:
Spiegelhalter_z = function(y, prob){
  alpha = 0.05
  z_score = sum((y-prob)*(1-2*prob))/sqrt(sum(((1-2*prob)^2)*prob*(1-prob)))
  print(z_score)
  if (abs(z_score) > qnorm(1-alpha/2)){
    print('reject null. NOT calibrated')
  } else {
    print('fail to reject. calibrated')
  }
  cat('z score: ', z_score, '\n')
  cat('p value: ', 1-pnorm(abs(z_score)), '\n')
    
  return(z_score)
}

Spiegelhalter_z(bin_data$default_flag_mean, bin_data$predicted_pd_mean)


[1] 0.01840817
[1] "fail to reject. calibrated"
z score:  0.01840817 
p value:  0.4926566 


[1] 0.01840817

### 5. ROC calculation

In [9]:
library(pROC, quietly = T)
roc_obj <- roc(test_data_main$default_flag, test_data_main$predicted_pd)
1 - auc(roc_obj)

Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following object is masked from 'package:BiocGenerics':

    var


The following objects are masked from 'package:stats':

    cov, smooth, var


Setting levels: control = 0, case = 1

Setting direction: controls > cases



[1] 0.5008548

### 6. CLAR

In [20]:
# https://www.rdocumentation.org/packages/VUROCS/versions/1.0/topics/clar

#' @title Cumulative LGD Accuracy Ratio
#' @description Calculates for a vector of realized categories \code{y} and a vector of predicted categories \code{hx} the cumulative LGD accuarcy ratio (CLAR) according to Ozdemir and Miu 2009.
#' @param hx a vector of  predicted values.
#' @param y a vector of  realized values.
#' @return The function returns the CLAR for a vector of realized categories \code{y} and a vector of predicted categories \code{hx}.
#' @examples clar(rep(1:5, each = 3), c(3, 3, 3, rep(2:5, each = 3)))
#' @references Ozdemir, B., Miu, P., 2009. Basel II Implementation. A Guide to Developing and Validating a Compliant Internal Risk Rating System. McGraw-Hill, USA.

clar <- function(y, hx) {
    if (any(is.na(hx)) | any(is.na(y))) {
        stop("\n both 'hx' and 'y' must not contain NA values")
    }

    if (length(hx) != length(y)) {
        stop("\n both 'hx' and 'y' must be of the same length")
    }

    nx <- length(hx)
    classes <- sort(union(unique(hx), unique(y)), decreasing = TRUE)
    num <- rep(NA, length(classes))
    for (i in 1:length(classes)) {
        num[i] <- sum(hx == classes[i])
    }
    cnum <- cumsum(num)
    index <- order(hx, decreasing = TRUE)
    hx <- hx[index]
    y <- y[index]
    corr <- rep(0, length(classes))
    for (i in 1:(length(classes) - 1)) {
        if (cnum[i] > 0) corr[i] <- sum(y[1:cnum[i]] >= classes[i])
    }
    corr <- corr / nx
    corr[length(classes)] <- 1
    obs <- cnum / nx

    res <- obs[1] * corr[1] / 2
    for (i in 2:length(corr)) {
        res <- res + (corr[i] - corr[i - 1]) * (obs[i] - obs[i - 1]) / 2 + (obs[i] - obs[i - 1]) * corr[i - 1]
    }
    res * 2
}

lgd_main <- read.csv("lgd_dataset.csv")

clar(lgd_main$realised_outcome, lgd_main$predicted_outcome)

[1] 0.84

In [78]:
lgd_main

realised_outcome,predicted_outcome
<int>,<int>
1,3
1,3
1,3
2,2
2,2
2,2
3,3
3,3
3,3


In [16]:
(rep(1:5,each=3))

[1] 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5

In [17]:
c(3,3,3,rep(2:5,each=3))

[1] 3 3 3 2 2 2 3 3 3 4 4 4 5 5 5

### 7. Loss Capture Ratio

In [ ]:
# https://github.com/selva86/InformationValue

### 8. Information Value

In [21]:
# --------------------------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See LICENSE.txt in the project root for license information.
# --------------------------------------------------------------------------------------------

#' @title
#' Calculate Weight of Evidence (WOE) and Information Value (IV) between a
#' single predictor and a single outcome variable.
#'
#' @description
#' Calculates Weight of Evidence (WOE) and Information Value (IV) between a
#' single predictor and a single outcome variable. This function implements the
#' common Information Value calculations whilst maintaining the minimum reliance
#' on external dependencies. Use `map_IV()` for the equivalent of
#' `Information::create_infotables()`, which performs calculations for multiple
#' predictors and a single outcome variable.
#'
#' @details
#' The approach used mirrors the one used in `Information::create_infotables()`.
#'
#' @param data Data frame containing the data.
#' @param outcome String containing the name of the outcome variable.
#' @param predictor String containing the name of the predictor variable.
#' @param bins Numeric value representing the number of bins to use.
#'
#' @import dplyr
#'
#' @return A data frame is returned as an output.
#'
calculate_IV <- function(data,
                         outcome,
                         predictor,
                         bins){

  pred_var <- data[[predictor]]
  outc_var <- data[[outcome]]

  # Check inputs
  if(sum(is.na(outc_var)) > 0){

    stop(
      glue::glue(
        "dependent variable {outcome} has missing values in the input training data frame"
      )
      )
  }

  # Compute q
  q <- stats::quantile(
    pred_var,
    probs = c(1:(bins - 1) / bins),
    na.rm = TRUE,
    type = 3
    )

  # Compute cuts
  cuts <- unique(q)

  # Compute intervals
  intervals <-
    findInterval(
      pred_var,
      vec = cuts,
      rightmost.closed = FALSE)

  # Compute cut_table
  cut_table <-
    table(
      intervals,
      outc_var) %>%
    as.data.frame.matrix()

  ## get min/max
  cut_table_2 <-
    data.frame(
    var = pred_var,
    intervals
  ) %>%
    group_by(intervals) %>%
    summarise(
      min = min(var, na.rm = TRUE) %>% round(digits = 1),
      max = max(var, na.rm = TRUE) %>% round(digits = 1),
      n = n(),
      .groups = "drop"
    ) %>%
    mutate(!!sym(predictor) :=
    glue::glue("[{round(min, digits = 1)},{round(max, digits = 1)}]")) %>%
    mutate(percentage = n / sum(n)) %>%
    select(!!sym(predictor), intervals, n, percentage)

  # Create variables that are double
  cut_table_1 <- as.numeric(cut_table$`1`)
  cut_table_0 <- as.numeric(cut_table$`0`)

  # Non-events in group
  n_non_event <- cut_table_1 * sum(cut_table_0) # t$y_1*sum_y_0
  n_yes_event <- cut_table_0 * sum(cut_table_1) # t$y_0*sum_y_1

  # Compute WOE

  cut_table_2$WOE <-
    ifelse(
      cut_table$`1` > 0 & cut_table$`0` > 0, # Both positive
      log(n_non_event / n_yes_event), # % of non-events divided by % of events
           0) # Otherwise impute 0

  # Compute IV_weight
  p1 <- cut_table$`1` / sum(cut_table$`1`)
  p0 <- cut_table$`0` / sum(cut_table$`0`)

  cut_table_2$IV_weight <- p1 - p0
  cut_table_2$IV <- cut_table_2$WOE * cut_table_2$IV_weight

  cut_table_2 %>%
    mutate(IV = cumsum(IV)) %>%
    # Maintain consistency with `Information::create_infotables()`
    select(
      !!sym(predictor),
      N = "n",
      Percent = "percentage",
      WOE,
      IV)
}

#' @title
#' Calculate Weight of Evidence (WOE) and Information Value (IV) between
#' multiple predictors and a single outcome variable, returning a list of
#' statistics.
#'
#' @description
#' This is a wrapper around `calculate_IV()` to loop through multiple predictors
#' and calculate their Weight of Evidence (WOE) and Information Value (IV) with
#' respect to an outcome variable.
#'
#' @details
#' The approach used mirrors the one used in `Information::create_infotables()`.
#'
#' @param data Data frame containing the data.
#' @param outcome String containing the name of the outcome variable.
#' @param predictors Character vector containing the names of the predictor
#'   variables. If `NULL` (default) is supplied, all numeric variables in the
#'   data will be used.
#' @param bins Numeric value representing the number of bins to use. Defaults to
#'   10.
#'
#' @import dplyr
#'
#' @return A list of data frames is returned as an output. The first layer of
#' the list contains `Tables` and `Summary`:
#'   -  `Tables` is a list of data frames containing the WOE and cumulative sum
#'   IV for each predictor.
#'   - `Summary` is a single data frame containing the IV for all predictors.
#'
map_IV <- function(data,
                   predictors = NULL,
                   outcome,
                   bins = 10){

  if(is.null(predictors)){

    predictors <-
      data %>%
      select(-!!sym(outcome)) %>%
      select(
        where(is.numeric)
      ) %>%
      names()
  }

  # List of individual tables
  Tables <-
    predictors %>%
    purrr::map(function(pred){

      calculate_IV(
        data = data,
        outcome = outcome,
        predictor = pred,
        bins = bins
      )
    }) %>%
    purrr::set_names(
      nm = purrr::map(
        .,
        function(df){
          names(df)[[1]]
        }
      )
    )

  # Compile Summary Table
  Summary <-
    list("df" = Tables,
         "names" = names(Tables)) %>%
    purrr::pmap(function(df, names){

      IV_final <-
        df %>%
        slice(nrow(df)) %>%
        pull(IV)

      data.frame(
        Variable = names,
        IV = IV_final
      )
    }) %>%
    bind_rows() %>%
    arrange(desc(IV))

  # Reorder and combine list
  c(
    list("Tables" = Tables[Summary$Variable]), # Reordered
    list("Summary" = Summary)
  )
}


pd_german_data <- read.csv("german_data.csv")


In [ ]:
# https://raw.githubusercontent.com/siyuany/ScoringModels/master/ScoringModels/R/woe.R

#' Weight of Evidence
#'
#' @description \code{woe} calculates the weight of evidence of each level in a
#' discrete variable with respect to a response.
#'
#' @param predictor a vector representing the predictor
#' @param response  a vecotr representing the response
#' @param df        a \code{data.frame} containing the predictor and the
#' response
#' @param x         the name of the predictor in \code{df}
#' @param y         the name of the response in \code{df}
#' @param good      the value representing 'good' in \code{response}, other
#' values representing bad.
#' @return a \code{data.frame} showing the WoE of each level, and encapsulated
#' as a \code{woe} object
#'
#' @examples
#' \dontrun{
#' library(ISLR)
#' # data.frame generic
#' woe(Default, 'student', 'default', good = 'No')
#' # default generic
#' woe(Default$student, Default$default, good = 'No')
#' SummaryWoe(Default, 'default', good = 'No')
#' }
#'
#' @export

woe <- function(...) {
    UseMethod('woe')
}

#' @describeIn woe default generic
#' @export
woe.default <- function(predictor, response, good = NA, ...) {
    y_levels <- sort(unique(response))
    if (length(y_levels) <= 1)
        stop("y is a vector of a unique value.")
    if (is.na(good)) {
        good <- y_levels[1]
    }
    if (!good %in% y_levels)
        stop('good does not exist in response')
    if (length(predictor) != length(response))
        stop('predictor should have the same length as response')
    include <- !(is.na(predictor) | is.na(response))
    response <- response[include]
    predictor <- predictor[include]

    y <- ifelse(response == good, 'Good', 'Bad')

    tbl <- tapply(y, predictor, function(z) {
        c(Good = sum(z == 'Good'),
          Bad  = sum(z == 'Bad'))
    })
    tbl <- as.data.frame(do.call(rbind, tbl), stringsAsFactors = FALSE)
    tbl <- rbind(tbl, Total = apply(tbl, 2, sum))
    tbl <- cbind(Levels = rownames(tbl), tbl)

    tbl$GoodR <- tbl$Good / tbl['Total', 'Good']
    tbl$BadR <- tbl$Bad / tbl['Total', 'Bad']
    tbl$WoE <- log(tbl$GoodR / tbl$BadR)
    tbl$IV <- (tbl$GoodR - tbl$BadR) * tbl$WoE
    tbl$IV[is.infinite(tbl$IV)] <- NA
    tbl$Percentage <- (tbl$Good + tbl$Bad) / sum(tbl['Total', c('Good', 'Bad')])
    tbl['Total', 'IV'] <- sum(tbl$IV, na.rm = TRUE)
    class(tbl) <- c('woe', class(tbl))
    rownames(tbl) <- NULL
    tbl
}

#' @describeIn woe generic for data.frame
#' @export
woe.data.frame <- function(df, x, y, ...) {
    if (!x %in% colnames(df))
        stop("x does not exist in df")
    if (!y %in% colnames(df))
        stop("y does not exist in df")

    woe.default(df[, x], df[, y], ...)
}


#' @rdname woe
#' @export
SummaryWoe <- function(df, y, ...) {
    predictors <- colnames(df)
    if (!any(y == predictors)) {
        stop("'y' does not exist in df")
    }

    predictors <- predictors[predictors != y]
    summary_woe <- list()
    for (predictor in predictors) {
        if (is.factor(df[, predictor])) {
            .woe <- woe(df, predictor, y, ...)
            summary_woe[[predictor]] <- list(woe = .woe,
                                             iv  = .woe[.woe$Levels == "Total",
                                                        "IV"])
        }

        if (is.numeric(df[, predictor])) {
            .cut <- bin(df, predictor, y, result = "cutpoints", ...)
            .woe <- woe(label.numeric(df[, predictor], .cut),
                        df[, y], ...)
            summary_woe[[predictor]] <- list(woe  = .woe,
                                             iv   = .woe[.woe$Levels == "Total",
                                                         "IV"],
                                             cutp = .cut)
        }
    }

    class(summary_woe) <- c('Summary.WoE', class(summary_woe))
    summary_woe
}

#' @rdname woe
#' @param summary_woe result given by SummaryWoe
#' @importFrom stringr str_length
#' @export
ReplaceWoe <- function(df, y, summary_woe) {
    columns <- colnames(df)
    if (!any(y == columns)) {
        stop("'y' does not exist in df")
    }
    if (class(summary_woe)[1] != 'Summary.WoE') {
        stop('summary_woe should be an object of Summary.WoE')
    }

    new_df <- as.data.frame(df[[y]], stringsAsFactors = FALSE)
    colnames(new_df) <- y
    predictors <- names(summary_woe)
    for (predictor in predictors) {
        if (!any(predictor == columns)) {
            stop(sprintf('%s does not exist in df', predictor))
        }
        n_char <- str_length(predictor)
        if (substr(predictor, n_char - 3, n_char) == '_woe') {
            warning(sprintf("%s ends with '_woe'. Skip!", predictor))
            next
        }

        .woe <- summary_woe[[predictor]][['woe']]
        .woe <- .woe[-nrow(.woe), ]
        if (is.factor(df[, predictor])) {
            new_df[, paste0(predictor, '_woe')] <-
                .woe[match(df[, predictor], .woe[, 'Levels']), 'WoE']
        }

        if (is.numeric(df[, predictor])) {
            cutp <- summary_woe[[predictor]][['cutp']]
            x_disc <- label.numeric(df[, predictor], cutp)
            new_df[, paste0(predictor, '_woe')] <-
                .woe[match(x_disc, .woe[, 'Levels']), 'WoE']
        }
    }

    new_df
}

In [37]:
res = map_IV (pd_german_data,
              predictors = c('amount','duration'),
              outcome='GoodCredit',
              bins = 10)

In [38]:
res$Summary

Variable,IV
<chr>,<dbl>
duration,0.2778772
amount,0.1117618


In [42]:
q = calculate_IV(pd_german_data,
                predictor='amount',
                outcome='GoodCredit',
                bins = 10)

In [46]:
q

amount,N,Percent,WOE,IV
<glue>,<int>,<dbl>,<dbl>,<dbl>
"[250,931]",99,0.099,0.06177736,0.0003824313
"[932,1258]",99,0.099,0.01438874,0.0004029866
"[1262,1474]",99,0.099,-0.23789141,0.0057272229
"[1478,1901]",102,0.102,-0.38665578,0.0197204795
"[1905,2315]",100,0.100,-0.04808619,0.0199494614
"[2319,2835]",100,0.100,-0.25131443,0.0259331383
"[2848,3578]",100,0.100,-0.36101335,0.0379669164
"[3590,4712]",100,0.100,-0.04808619,0.0381958983
"[4716,7166]",100,0.100,0.31508105,0.0486985998


In [47]:
sum(q$IV)

[1] 0.3087389

In [ ]:
# https://www.r-bloggers.com/2013/07/r-credit-scoring-woe-information-value-in-woe-package/

In [76]:
#' Calculate Information Value for character or factor columns
#'
#' This function calculates information value for character or factor columns of data frame. 
#'
#' @param df data frame with at least two columns (predictor x and outcome y)
#' @param x column (character or factor) for which Information Value should be calculated
#' @param y column (integer) with binary outcome. y has to be a column in df data frame. It is suggested that y is factor with two levels "bad" and "good" If there are no levels good/bad than the following assumptions are applied - if y is integer, than 0=good and 1=bad. If y is factor than level 2 is assumed to mean bad and 1 good.
#' @param verbose Prints additional details when TRUE. Useful mainly for debugging.
#' @export
#' @examples
#' iv.str(german_data,"purpose","gb")
#' iv.str(german_data,"savings","gb")

iv.str <- function(df,x,y,verbose=FALSE) {
  if (!(class(df)=="data.frame")) {
    stop("Parameter df has to be a data frame.")
  } 
  if (!(is.character(df[, x]) || is.factor(df[, x]))) {
    stop(paste("Input is not a character or factor! Variable:", x))
    } 
  if (!(is.numeric(df[, y]) || is.factor(df[, y]))) {
    stop("Outcome is not a number nor factor!")
  } 
  if (length(unique(df[, y])) != 2) {
    if(verbose) paste(cat(unique(df[,y])),"\n")
    stop("Not a binary outcome")
    }
  if (!(all(sort(unique(df[, y])) == c(0,1))) && is.numeric(df[,y])) {
    stop("Numeric outcome has to be encoded as 0 (good) and 1 (bad). \n")
  }
  if (is.factor(df[,y]) && all(levels(df[,y])[order(levels(df[,y]))]==c("bad","good"))) {
    if (verbose) cat("Assuming good = level 'good' and bad = level 'bad' \n")
    total_1 <- sum(df[,y]=="bad")
    } else if (is.factor(df[,y])) {
    if (verbose) cat("Factor: Assuming bad = level 2 and good = level 1 \n")
    total_1 <- sum(as.integer(df[, y])-1)

    } else {
    if (verbose) cat("Numeric: Assuming bad = 1 and good = 0 \n")
    total_1 <-sum(df[, y])

  }

  outcome_0 <- outcome_1 <- NULL # This is needed to avoid NOTES about not visible binding from R CMD check
  
  total_0 <- nrow(df) - total_1      
  iv_data <- data.frame(unclass(table(df[, x],df[, y])))
  
  if (all(names(iv_data)==c("bad","good"))) {
    iv_data <- iv_data[,c(2,1)]
  }


  names(iv_data) <- c("outcome_0","outcome_1")
  iv_data <-  within(iv_data, {
                class <- row.names(iv_data)
                variable <- x
                pct_0 <- outcome_0 / total_0
                pct_1 <- outcome_1 / total_1
                odds <-  pct_0 / pct_1
                woe <- log(odds)
                miv <- (pct_0 - pct_1) * woe    
  })

  if(is.factor(df[,x])) {
    iv_data$class <- factor(iv_data$class,levels=levels(df[,x]))
  }  
  
  iv_data <- iv_data[c("variable","class","outcome_0","outcome_1","pct_0","pct_1","odds","woe","miv")]

  if(any(iv_data$outcome_0 == 0) | any(iv_data$outcome_1 == 0)) {
    warning("Some group for outcome 0 has zero count. This will result in -Inf or Inf WOE. Replacing - ODDS=1, WoE=0, MIV=0. \n The bin is either too small or suspiciously predictive. \n You should fix this before running any model. It does not make any sense to keep WoE = 0 for such bin.")
      iv_data$woe <- ifelse(is.infinite(iv_data$woe),0,iv_data$woe)
      iv_data$miv <- ifelse(is.infinite(iv_data$miv),0,iv_data$miv)
      iv_data$odds <-ifelse(is.infinite(iv_data$odds),1,iv_data$odds)
  }
  
  rownames(iv_data) <- NULL
  cat (paste("Information Value",round(sum(iv_data$miv),10),"\n"))
  iv_data
}


In [77]:
pd_german_data <- read.csv("german_data.csv")
iv.str(pd_german_data, 'checkingstatus', 'GoodCredit', verbose=TRUE)

Numeric: Assuming bad = 1 and good = 0 
Information Value 0.6660115034 


variable,class,outcome_0,outcome_1,pct_0,pct_1,odds,woe,miv
<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
checkingstatus,A11,139,135,0.1985714,0.45000000,0.4412698,-0.8180987,0.205693389
checkingstatus,A12,164,105,0.2342857,0.35000000,0.6693878,-0.4013918,0.046446763
checkingstatus,A13,49,14,0.0700000,0.04666667,1.5000000,0.4054651,0.009460853
checkingstatus,A14,348,46,0.4971429,0.15333333,3.2422360,1.1762632,0.404410499


In [ ]:
(data, "checkingstatus", "GoodCredit")

### 9. LGD t-test

### 10. Migration matrix stability

### 11. PSI

In [49]:
#' @title
#' Population Stability Index
#' @description
#' \code{psi} calculates the popolation stability index.
#' @author Siyuan Yao
#' @references \href{What is population stability index? - Quora}{https://www.quora.com/What-is-population-stability-index}
#'
#' @param original The original set of a measurement, should be a factor or numeric
#' @param current  The current set of a measurement, should be a factor or numeric
#' @param cut.points It won't work if original and current are factors, and it cannot be NULL if original and current are numerical. This function uses this argument to bin \code{original} and \code{current} with left-closed right-open intervals.
#' @param cut.levels specifies how many levels to split a numeric variable. If \code{cut.points} is provided, this parameter will be ignored. When using this parameter, a numeric variable will be split using \code{cut}.
#'
#' @return a \code{psi} object
#'
#' @details psi measures the stablity of the population. Usually we can believe the population stays the same as the past if psi is less than 0.1, and a significant shift can be recognised if psi is greater than 0.25. The outcome of this function is a numeric, with details stored as attributes. You can use \code{summary} function to see all of the detailed information. Fot the situation where some of the levels has no element in either original population or current population and the psi does not exist for such levels, the empty levels will not be taken into account and a warning will inform you of this. Again, by using \code{summary} you could know everything inside.
#'
#' @examples
#' data("iris")
#' train <- sample(nrow(iris), nrow(iris) * .7)
#' train.species <- iris$Species[train]
#' test.species <- iris$Species[-train]
#' p <- psi(train.species, test.species)
#' p
#' summary(p)
#'
#' @import magrittr
#' @export
psi <- function(original,
                current,
                cut.points = NULL,
                cut.levels = ifelse(is.null(cut.points), 5L, NULL)) {

    if (!is.null(cut.points)) {
        cut.levels <- NULL
    }

    if (!is.null(cut.levels)) {
        if (cut.levels < 3) {
            warning("cut.levels must be an interger greater than 3")
            cut.levels <- 3L
        }
    }

    # binning numeric
    label.numeric <- function(x, cuts, na.level = NULL) {
        cuts <- sort(cuts)
        n_cut <- length(cuts)
        level.names <- paste0('<= ', cuts) %>% c(paste0('> ', cuts[n_cut]))

        if (any(is.null(na.level))) {
            na.level <- any(is.na(x))
        }
        if (na.level) level.names <- c('Missing', level.names)
        else {
            if(any(is.na(x))) stop('x has NA value while na.level is FALSE')
        }

        y <- vector('integer', length(x))

        for (i in n_cut:1) {
            y[x <= cuts[i]] <- i
        }
        y[x > cuts[n_cut]] <- n_cut + 1

        if (na.level) {
            y <- level.names[y + 1]
            y[is.na(x)] <- 'Missing'
        } else {
            y <- level.names[y]
        }

        factor(y, level.names)
    }


    # try to convert original & current to factor
    if (is.numeric(original) & is.numeric(current)) {
        if (is.null(cut.points)) {
            cut.points <- unname(quantile(original,
                                          seq(0, 1, length.out = cut.levels),
                                          type = 1,
                                          na.rm = TRUE))
            cut.points <- cut.points[-c(1, length(cut.points))]
        }
        na.level <- any(is.na(c(original, current)))
        # attr(res, 'original.num') <- original
        # attr(res, 'current.num') <- current
        original <- label.numeric(original, cut.points, na.level)
        current  <- label.numeric(current, cut.points, na.level)
    }

    if (!is.factor(original) | !is.factor(current)) {
        stop('original and current should be numeric or factor simultaneously.')
    }
    if (any(levels(original) != levels(current))) {
        common_lv <- union(levels(original), levels(current))
        original <- factor(original, levels = common_lv)
        current  <- factor(current,  levels = common_lv)

    }


    levels.name <- levels(original)
    org.stat.tbl <- tapply(X = original,
                           INDEX = original,
                           FUN = length,
                           simplify = TRUE) %>%
        sapply(function(x) ifelse(is.na(x), 0, x))
    cur.stat.tbl <- tapply(X = current,
                           INDEX = current,
                           FUN = length,
                           simplify = TRUE)%>%
        sapply(function(x) ifelse(is.na(x), 0, x))

    tbl <- data.frame(Levels = levels.name,
                      OrgCnt = org.stat.tbl,
                      CurCnt = cur.stat.tbl,
                      OrgPct = org.stat.tbl / sum(org.stat.tbl),
                      CurPct = cur.stat.tbl / sum(cur.stat.tbl))
    tbl$Index <- (tbl$CurPct - tbl$OrgPct) * log(tbl$CurPct / tbl$OrgPct)

    psi <- sum(tbl$Index[tbl$OrgCnt != 0 & tbl$CurCnt != 0])
    res <- psi
    tbl <- rbind(tbl, data.frame(Levels = 'Total',
                                 OrgCnt = sum(org.stat.tbl),
                                 CurCnt = sum(cur.stat.tbl),
                                 OrgPct = 1,
                                 CurPct = 1,
                                 Index = psi))
    rownames(tbl) <- NULL

    tbl$OrgPct <- round(tbl$OrgPct, 4)
    tbl$CurPct <- round(tbl$CurPct, 4)
    tbl$Index  <- round(tbl$Index,  4)

    attr(res, 'tbl') <- tbl
    # attr(res, 'original') <- original
    # attr(res, 'current')  <- current
    if (any(tbl$OrgCnt == 0 | tbl$CurCnt == 0)) {
        attr(res, 'Empty Levels') <- tbl$Levels[tbl$OrgCnt == 0 | tbl$CurCnt == 0] %>% as.character()
        warning('Some of the levels are empty, and PSI may be inaccurate. Please use `summary` to see the details.')
    }
    class(res) <- c('psi', 'numeric')
    res
}


#' @export
print.psi <- function(x, ...) {
    cat('PSI :', round(x, 4), '\n')
    # NextMethod('print')
}

#' @export
summary.psi <- function(object, ...) {
    cat('PSI:', round(object, 4), '\n\n')
    print(attr(object, 'tbl'))

    if (!is.null(attr(object, 'Empty Levels'))) {
        cat('\nEmpty Levels: ', paste0(attr(object, 'Empty Levels'), collapse = ', '), '\n')
    }
    # NextMethod('summary')
}

PSI : 0.0243

PSI: 0.0243 

      Levels OrgCnt CurCnt OrgPct CurPct  Index
1     setosa     37     13 0.3524 0.2889 0.0126
2 versicolor     33     17 0.3143 0.3778 0.0117
3  virginica     35     15 0.3333 0.3333 0.0000
4      Total    105     45 1.0000 1.0000 0.0243


In [62]:
# create dataset
pd_psi <- read.csv("test_pd_psi.csv")

expected <- pd_psi$remaining_mat_bin[pd_psi$year_bins=='period_1']
actual <- pd_psi$remaining_mat_bin[pd_psi$year_bins=='period_2']
p <- psi(expected, actual, cut.points = c(0,1,2,3,4,5,6,7,8))
summary(p)

### 12. Bayesian Error Rate

In [ ]:
`bayes.error.rate` <-
function(tar, non, lpo) {
  n <- length(lpo)
  pmiss <- pfa <- vector(mode="numeric", length=n)
  for (i in 1:n) {
    p <- sigmoid(tar+lpo[i])
    pmiss[i] <- mean(1-sign(p-0.5))/2
    p <- sigmoid(non+lpo[i])
    pfa[i] <- mean(1-sign(0.5-p))/2
  }
  pmiss*sigmoid(lpo) + pfa*sigmoid(-lpo)
}

### 13. SomersD

In [92]:
library(DescTools)
SomersDelta(x=c(0, 1, 1, 1, 1), y=c(1, 1, 1, 0, 1), direction = 'column')

[1] -0.25

#### 14. Kendall Tau

In [93]:
x<-c(1,2,3,2,1,3,4,2,5,2,6,5,5)
y<-c(5,5,6,2,1,4,4,2,1,2,1,5,5)
res<-cor.test(x,y, method="kendall")
res


	Kendall's rank correlation tau

data:  x and y
z = 0.12931, p-value = 0.8971
alternative hypothesis: true tau is not equal to 0
sample estimates:
       tau 
0.03030651 


### 15. Spearman correlation

In [96]:
x = c(1,2,3,4,5)
y = c(5,6,7,8,7)
 
# Calculating
# Correlation coefficient
# Using cor() method
cor(x, y, method = "spearman")


[1] 0.8207827

### TO BE DELETED

In [10]:
brier.score

function (predictedArr, truthArr) 
{
    if (length(predictedArr) != length(truthArr)) {
        print("ERROR: length NOT equal!!")
    }
    temp.vec <- (truthArr - predictedArr)^2
    sum(temp.vec)
}
<bytecode: 0x00000000330ca708>
<environment: namespace:iterativeBMA>

In [11]:
pp.testing

function (rating.label, pdc, no, nb, alpha = 0.05) 
{
    l <- list(rating.label, pdc, no, nb)
    l.check <- !length(table(sapply(l, length))) == 1
    if (l.check) {
        stop("arguments rating.label, pdc, no and nb have to be of the same length.")
    }
    if (any(!(is.numeric(pdc) | is.numeric(n) | is.numeric(nb)))) {
        stop("All arguments have to of numeric type.")
    }
    if (any(pdc > 1 | pdc < 0)) {
        stop("pdc and odr have to be between 0 and 1.")
    }
    if (any(nb > no)) {
        stop("Number of defaults cannot be greater than number of observations.")
    }
    if (any(nb < 0 | no < 0)) {
        stop("no and nb arguments cannot be negative.")
    }
    if (alpha > 1 | alpha < 0) {
        stop("significance level (alpha) has to be between 0 and 1.")
    }
    bino.test <- pbinom(nb - 1, no, pdc, lower.tail = FALSE)
    bino.test.res <- ifelse(bino.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    jeff.test <- pbeta(pdc, nb + 0.5, no - nb + 0.5)
    jeff.test.res <- ifelse(jeff.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    odr <- nb/no
    zsco.test <- zscore.test(pdc = pdc, odr = odr, no = no)
    zsco.test.res <- ifelse(zsco.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    if (length(pdc) == 1) {
        hosm.test <- NA
        hosm.test.res <- "Only one rating grade."
    }
    else {
        hosm.test <- hl.test(pdc = pdc, no = no, nb = nb)
        hosm.test.res <- ifelse(hosm.test >= alpha, "H0: ODR <= PDC", 
            "H1: ODR > PDC")
    }
    res <- data.frame(rating = rating.label, no = no, nb = nb, 
        odr = odr, pdc = pdc, alpha = alpha, binomial = bino.test, 
        binomial.res = bino.test.res, jeffreys = jeff.test, jeffreys.res = jeff.test.res, 
        zscore = zsco.test, zscore.res = zsco.test.res, hosmer.lemeshow = hosm.test, 
        hosmer.lemeshow.res = hosm.test.res)
    return(res)
}
<bytecode: 0x000000002ba1f778>
<environment: namespace:PDtoolkit>

In [12]:
hl.test <- function(pdc, no, nb) {
	k <- length(no)
	hl <- sum((nb - no * pdc)^2 / (no * pdc * (1 - pdc)), na.rm = T)
	p.val <- pchisq(q = hl, df = k, lower.tail = FALSE)
return(p.val)
}